<a href="https://colab.research.google.com/github/AyushiKashyapp/foodwise_knowledgeDB/blob/main/Neo4jGraphCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xlrd
!pip install neo4j-driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j-driver: filename=neo4j_driver-5.21.0-py3-none-any.whl size=287063 sha256=09db88aa97cf664aa835f72e957d07c22564557a6217a1e7185b137f44f370df
  Stored in directory: /root/.cache/pip/wheels/d8/c1/3d/70a5f1476660d706a60642907c0cc2cffa986018afd29fc60b
Successfully built neo4j-driver


In [2]:
import pandas as pd
import re

In [3]:
triples = pd.read_excel('triples_data.xlsx')
triples.head()

,chunk_id,head,type,tail
0,Chunk_1,IFA2024,country,Singapore
1,Chunk_1,IFA2024,location,Singapore
2,Chunk_1,IFA2024 in Singapore,country,Singapore
3,Chunk_1,Jam Jar,subclass of,jam jar
4,Chunk_1,oak tree,has parts of the class,leaves


In [4]:
def sanitize_relation_type(relation):
    # Replace any non-alphanumeric characters with underscores
    return re.sub(r'\W+', '_', relation)

In [7]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "bolt://3.83.136.242:7687",
  auth=basic_auth("neo4j", "firings-lock-conflicts"))

def create_knowledge_graph(triples):
    with driver.session() as session:
        for _, triple in triples.iterrows():
            head = triple['head']
            relation = sanitize_relation_type(triple['type'])
            tail = triple['tail']

            cypher_query = f"""
            MERGE (h:Entity {{name: $head}})
            MERGE (t:Entity {{name: $tail}})
            MERGE (h)-[:{relation.upper()}]->(t)
            """

            session.run(cypher_query, head=head, tail=tail)


In [8]:
create_knowledge_graph(triples)

In [9]:
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "bolt://3.83.136.242:7687",
  auth=basic_auth("neo4j", "firings-lock-conflicts"))

cypher_query = '''
MATCH (n)
RETURN COUNT(n) AS count
LIMIT $limit
'''

with driver.session(database="neo4j") as session:
  results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      limit=10).data())
  for record in results:
    print(record['count'])

driver.close()

<ipython-input-9-cfa5160c8f9a>:14: DeprecationWarning: read_transaction has been renamed to execute_read
  results = session.read_transaction(


577


In [10]:
from neo4j import GraphDatabase, basic_auth

# Establish connection
driver = GraphDatabase.driver(
    "bolt://3.83.136.242:7687",
    auth=basic_auth("neo4j", "firings-lock-conflicts")
)

def get_relations_where_head_is_enda(driver):
    query = """
    MATCH (h:Entity {name: 'Enda Kenny'})-[r]->(t:Entity)
    RETURN h.name AS head, TYPE(r) AS relation, t.name AS tail
    """

    relations = []
    with driver.session() as session:
        result = session.run(query)
        for record in result:
            relations.append({
                'head': record['head'],
                'relation': record['relation'],
                'tail': record['tail']
            })

    return relations

# Execute the query and get the relations
relations_where_head_is_enda = get_relations_where_head_is_enda(driver)

# Print the relations
print("Relations where 'Enda Kenny' is the head:")
for relation in relations_where_head_is_enda:
    print(relation)

# Close the driver connection
driver.close()

Relations where 'Enda Kenny' is the head:
{'head': 'Enda Kenny', 'relation': 'CANDIDACY_IN_ELECTION', 'tail': '2011 general election'}
{'head': 'Enda Kenny', 'relation': 'CANDIDACY_IN_ELECTION', 'tail': '2002 general election'}
{'head': 'Enda Kenny', 'relation': 'SPOUSE', 'tail': 'Fionnuala OKelly'}
{'head': 'Enda Kenny', 'relation': 'CANDIDACY_IN_ELECTION', 'tail': '2011 election campaign'}
{'head': 'Enda Kenny', 'relation': 'NOTABLE_WORK', 'tail': 'Second national address'}
{'head': 'Enda Kenny', 'relation': 'POSITION_HELD', 'tail': 'European Commission president'}
{'head': 'Enda Kenny', 'relation': 'POSITION_HELD', 'tail': 'European Council president'}
{'head': 'Enda Kenny', 'relation': 'POSITION_HELD', 'tail': 'Prime Minister'}
{'head': 'Enda Kenny', 'relation': 'CANDIDACY_IN_ELECTION', 'tail': '2007 election'}
{'head': 'Enda Kenny', 'relation': 'EMPLOYER', 'tail': 'TG4'}
{'head': 'Enda Kenny', 'relation': 'EMPLOYER', 'tail': 'RTÉ'}
{'head': 'Enda Kenny', 'relation': 'EMPLOYER', 't

In [16]:
from neo4j import GraphDatabase, basic_auth

# Establish connection
driver = GraphDatabase.driver(
    "bolt://3.83.136.242:7687",
    auth=basic_auth("neo4j", "firings-lock-conflicts")
)

def get_all_food_wise_relations(driver):
    query = """
    MATCH (n)-[r]->(m)
    WHERE n.name CONTAINS 'Food Wise' OR m.name CONTAINS 'Food Wise' OR TYPE(r) CONTAINS 'Food Wise'
    RETURN n AS node1, r AS relation, m AS node2
    UNION
    MATCH (n)
    WHERE n.name CONTAINS 'Food Wise'
    RETURN n AS node1, null AS relation, null AS node2
    """

    relations = []
    with driver.session() as session:
        result = session.run(query)
        for record in result:
            node1 = record['node1']
            relation = record['relation']
            node2 = record['node2']
            relations.append({
                'node1': node1,
                'relation': relation,
                'node2': node2
            })

    return relations

# Execute the query and get the relations
food_wise_relations = get_all_food_wise_relations(driver)

# Print the relations
print("Relations containing 'Food Wise':")
for relation in food_wise_relations:
    print(relation)

# Close the driver connection
driver.close()


Relations containing 'Food Wise':
